In [3]:
import requests
import time, os
import random
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML
from collections import defaultdict
import csv
import pandas as pd

In [168]:
base_url = 'https://www.pro-football-reference.com/players/'

#scrape Jared Goff first
#player = 'players/G/GoffJa00.htm'

In [221]:
def get_player_htm():
    '''
    Use name to get name for player's html page identifier
    return:
        dict of {first_part_htm_names : player_name}
    '''
    qb_list_file = './pfr_scraped/master_qb_list.csv'
    qb_df = pd.read_csv(qb_list_file, index_col=0)
    htm_name_dict = {}

    qb_df = qb_df['0'].str.split(n=1, expand=True).copy()
    qb_df[2] = qb_df[0] + ' ' + qb_df[1]
    qb_df.rename({0:'First', 1:'Last', 2:'Full'}, axis=1, inplace=True)
    qb_df['htm_name'] = qb_df['Last'].str.slice(stop=4) + qb_df['First'].str.slice(stop=2)
    htm_name_dict = dict(zip(qb_df['htm_name'], qb_df['Full']))
    return htm_name_dict

In [222]:
htm_name_dict = get_player_htm()
htm_name_dict

{'MannPe': 'Peyton Manning',
 'BreeDr': 'Drew Brees',
 'BradSa': 'Sam Bradford',
 'PalmCa': 'Carson Palmer',
 'SchaMa': 'Matt Schaub',
 'RyanMa': 'Matt Ryan',
 'RivePh': 'Philip Rivers',
 'MannEl': 'Eli Manning',
 'SancMa': 'Mark Sanchez',
 'OrtoKy': 'Kyle Orton',
 'BradTo': 'Tom Brady',
 'HennCh': 'Chad Henne',
 'FlacJo': 'Joe Flacco',
 'RodgAa': 'Aaron Rodgers',
 'FreeJo': 'Josh Freeman',
 'McNaDo': 'Donovan McNabb',
 'CassMa': 'Matt Cassel',
 'HassMa': 'Matt Hasselbeck',
 'FitzRy': 'Ryan Fitzpatrick',
 'CutlJa': 'Jay Cutler',
 'HillSh': 'Shaun Hill',
 'RoetBe': 'Ben Roethlisberger',
 'VickMi': 'Michael Vick',
 'GarrDa': 'David Garrard',
 'FavrBr': 'Brett Favre',
 'SmitAl': 'Alex Smith',
 'CampJa': 'Jason Campbell',
 'AndeDe': 'Derek Anderson',
 'KitnJo': 'Jon Kitna',
 'ClauJi': 'Jimmy Clausen',
 'CollKe': 'Kerry Collins',
 'McCoCo': 'Colt McCoy',
 'RomoTo': 'Tony Romo',
 'KolbKe': 'Kevin Kolb',
 'YounVi': 'Vince Young',
 'SmitTr': 'Troy Smith',
 'MoorMa': 'Matt Moore',
 'StafMa': 'M

In [223]:
def scrape_page(htm_dict):
    '''
    Scrape page and store in a list of soup files
    arg:
        list of qb's name as html identifier
    return:
        list of players pages in soup form
    '''
    ua = UserAgent()
    user_agent = {'User-agent' : ua.random}
    player_page_soups = []
    counter = 0
    
    for qb_htm, qb_name in htm_dict.items():
        for x in range(6):
            position_list = []
            name = qb_htm + '0' + str(x) + '.htm'
            add_url = qb_htm[0] + '/' + name
            response = requests.get(base_url + add_url, headers=user_agent)
            if response.status_code == 200:
                print(str(response.status_code) + ' ' + add_url)
                page_soup = BeautifulSoup(response.text, 'lxml')
                player_name = page_soup.find('h1', {'itemprop':'name'}).text
                position = page_soup.find_all('td',{'data-stat':'pos'})
                for pos in position:
                    position_list.append(pos.text.lower())
                if (player_name.strip() != qb_name.strip()) or ('qb' not in position_list):
                    time.sleep(random.random() + random.randint(1,4))
                    continue
                else:
                    break
            else:
                print('Something\'s wrong with:' + add_url)
                break
        player_page_soups.append(page_soup)
        time.sleep(random.random() + random.randint(1,4))        
    return player_page_soups

In [224]:
player_page_soups = scrape_page(htm_name_dict)

200 M/MannPe00.htm
200 B/BreeDr00.htm
200 B/BradSa00.htm
200 P/PalmCa00.htm
200 S/SchaMa00.htm
200 R/RyanMa00.htm
200 R/RivePh00.htm
200 M/MannEl00.htm
200 S/SancMa00.htm
200 O/OrtoKy00.htm
200 B/BradTo00.htm
200 H/HennCh00.htm
200 H/HennCh01.htm
200 F/FlacJo00.htm
200 R/RodgAa00.htm
200 F/FreeJo00.htm
200 M/McNaDo00.htm
200 C/CassMa00.htm
200 H/HassMa00.htm
200 F/FitzRy00.htm
200 C/CutlJa00.htm
200 H/HillSh00.htm
200 R/RoetBe00.htm
200 V/VickMi00.htm
200 G/GarrDa00.htm
200 F/FavrBr00.htm
200 S/SmitAl00.htm
200 S/SmitAl01.htm
200 S/SmitAl02.htm
200 S/SmitAl03.htm
200 C/CampJa00.htm
200 A/AndeDe00.htm
200 K/KitnJo00.htm
200 C/ClauJi00.htm
200 C/CollKe00.htm
200 M/McCoCo00.htm
200 R/RomoTo00.htm
200 K/KolbKe00.htm
200 Y/YounVi00.htm
200 S/SmitTr00.htm
200 S/SmitTr01.htm
200 S/SmitTr02.htm
200 M/MoorMa00.htm
200 M/MoorMa01.htm
200 S/StafMa00.htm
200 N/NewtCa00.htm
200 D/DaltAn00.htm
200 G/GrosRe00.htm
200 J/JackTa00.htm
200 G/GabbBl00.htm
200 P/PondCh00.htm
200 S/SkelJo00.htm
200 T/TeboTi

In [189]:
def get_player_data(soup):
    '''
    Get player's name (string) and stat table in bs resultset object
    arg:
        BS object of player's web page
    return:
        list containing [player_name, bs4.resultset]
    '''
    player_data = []
    name = soup.find('h1', {'itemprop':'name'}).text.strip()
    big_table = soup.find_all('table', {'id':'passing'})
    return name, big_table

In [177]:
def get_stats(player_data):
    name = player_data[0]
    stat_table = player_data[1]
    annual_stat_dict = defaultdict(list)
    player_stat_list = []

    for ele in stat_table:
        stat_block = ele.find_all('tr', class_='full_table')
        for stats_rows in stat_block:
            year = int(stats_rows.find('a').text)
            stats = stats_rows.find_all('td')
            for x in stats:
                annual_stat_dict[year].append(x.text)
    player_stat_list.append(name)
    player_stat_list.append(annual_stat_dict)
    return player_stat_list

In [87]:
stats_for_df = get_stats(player_data)

In [178]:
def get_headers(player_data):
    '''
    Grab stat headers from player's BS table
    arg:
        BS object containing players info (from scrape_page)
    return:
        stat labels in a list
    '''
    table = player_data[1]
    stat_header = ['name']

    stat_table = table[0].find_all('tr')
    header_soup = stat_table[0].find_all('th')
    for header in header_soup:
        stat_header.append(header.text.strip())
    return stat_header

In [77]:
headers = get_headers(player_data)
len(headers)

33

In [88]:
stats_for_df

['Carson Palmer',
 defaultdict(list,
             {2004: ['25',
               'CIN',
               'QB',
               '9',
               '13',
               '13',
               '6-7-0',
               '263',
               '432',
               '60.9',
               '2897',
               '18',
               '4.2',
               '18',
               '4.2',
               '140',
               '76',
               '6.7',
               '5.7',
               '11.0',
               '222.8',
               '77.3',
               '',
               '25',
               '178',
               '5.95',
               '4.96',
               '5.5',
               '2',
               '3',
               '8'],
              2005: ['26',
               'CIN',
               'QB',
               '9',
               '16',
               '16',
               '11-5-0',
               '345',
               '509',
               '67.8',
               '3836',
               '32',
               

In [179]:
def create_player_df(stats_for_df, headers):
    agg_stat_list = []

    for key, value in stats_for_df[1].items():
        list_for_df = []
        list_for_df.append(stats_for_df[0])
        list_for_df.append(key)
        for x in value:
            list_for_df.append(x)
        agg_stat_list.append(list_for_df)
    player_df = pd.DataFrame(agg_stat_list,columns=headers)
    return player_df

In [225]:
def create_raw_stat_df(player_page_soups, pkl=False):
    master_raw_df = pd.DataFrame()
    error_list = []
    for player_soup in player_page_soups:
        player_data = get_player_data(player_soup)
        if not player_data[1]:
            error_list.append(player_data[0])
            pass
        else:
            headers = get_headers(player_data)
            player_stats = get_stats(player_data)
            player_df = create_player_df(player_stats, headers)
        master_raw_df = master_raw_df.append(player_df)
        if pkl:
            master_raw_df.to_pickle('./Pickles/master_raw_df.pkl')
    return master_raw_df, error_list

In [229]:
master_raw_df = create_raw_stat_df(player_page_soups, pkl=True)

In [230]:
master_raw_df[0]

,name,Year,Age,Tm,Pos,No.,G,GS,QBrec,Cmp,...,Rate,QBR,Sk,Yds,NY/A,ANY/A,Sk%,4QC,GWD,AV
0,Peyton Manning,1998,22,IND,QB,18,16,16,3-13-0,326,...,71.2,,22,109,6.08,4.84,3.7,1,1,11
1,Peyton Manning,1999,23,IND,QB,18,16,16,13-3-0,331,...,90.7,,14,116,7.35,7.06,2.6,6,7,18
2,Peyton Manning,2000,24,IND,QB,18,16,16,10-6-0,357,...,94.7,,20,131,7.25,7.22,3.4,2,3,18
3,Peyton Manning,2001,25,IND,QB,18,16,16,6-10-0,343,...,84.1,,29,232,6.77,5.88,5.0,,,15
4,Peyton Manning,2002,26,IND,QB,18,16,16,10-6-0,392,...,88.8,,23,145,6.60,6.09,3.7,4,5,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,Nick Mullens,2019,24,SFO,,4,1,0,,0,...,,,0,0,,,,,,0
0,Jeff Driskel,2018,25,CIN,qb,6,9,5,1-4-0,105,...,82.2,31.1,16,122,4.59,4.74,8.3,,,2
1,Jeff Driskel,2019,26,DET,,2,3,3,0-3-0,62,...,75.3,,11,46,5.51,4.65,9.5,,,3
0,Lamar Jackson,2018,21,BAL,qb,8,16,7,6-1-0,99,...,84.5,46.3,16,71,6.08,5.99,8.6,0,1,8


In [209]:
err_qb_list

['Tremon Smith', 'Robert Griffin', 'Chris White']